In [1]:
#import all packages necessary for the program

import numpy as np          #classics
import pandas as pd
import matplotlib as plt      
import geopandas as gpd

import ee                   #earth engine
import geemap
import geeet
from geeet.eepredefined import landsat

import time                 #time management

#initialize earth engine
#ee.Authenticate()           #uncomment this line if you need to authenticate your earth engine account
ee.Initialize()

In [3]:

# =========================
# USER PARAMETERS
# =========================
region = ee.FeatureCollection("projects/ee-redwall6152/assets/DC_bound")
start_year = 2013
end_year = 2025
growing_months = range(4, 11)  # April–October
max_cloud_fraction = 25  # percent
export_folder = "GEEET_monthly"  # Parent folder in Google Drive

# =========================
# SUPPORT FUNCTIONS
# =========================

def add_cloud_cover_roi(img):
    """Add % cloud cover within region as image property."""
    qa = img.select('QA_PIXEL')
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))

    total = cloud_mask.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=region.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('QA_PIXEL')

    clear = cloud_mask.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region.geometry(),
        scale=30,
        maxPixels=1e9
    ).get('QA_PIXEL')

    cloud_fraction = ee.Number(1).subtract(ee.Number(clear).divide(ee.Number(total))).multiply(100)
    return img.set('CLOUD_COVER_ROI', cloud_fraction)


def mask_clouds(img):
    """Mask only clouds based on QA_PIXEL band."""
    qa = img.select('QA_PIXEL')
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return img.updateMask(cloud_mask)

def mask_snow_et(img):
    """Set ET = 0 where snow is present (do NOT mask)."""
    qa = img.select('QA_PIXEL')
    snow_mask = qa.bitwiseAnd(1 << 5).neq(0)

    et = img.select('ET')
    et_fixed = et.where(snow_mask, 0)

    return img.addBands(et_fixed.rename('ET'), overwrite=True)


def clip_to_region(img):
    """Clip image to the study region."""
    return img.clip(region)


def filter_landsat_8_9(collection):
    """Filter to Landsat 8/9 scenes and acceptable cloud cover."""
    return collection.filter(ee.Filter.inList('SPACECRAFT_ID', ['LANDSAT_8', 'LANDSAT_9'])) \
                     .filter(ee.Filter.lte('CLOUD_COVER_ROI', max_cloud_fraction))

# =========================
# ET WORKFLOW
# =========================
# Define your TSEB + LE extrapolation workflow
workflow = [
    geeet.tseb.tseb_series,
    landsat.extrapolate_LE ] # adds "ET" band (mm/day)

def get_monthly_mean_et(year, month):
    """Compute smoothed, median monthly ET composite for one month."""
    year = ee.Number(year)
    month = ee.Number(month)
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    # --- Generate ET collection for this month ---
    col = landsat.mapped_collection(
        workflow,
        date_start=start_date,
        date_end=end_date,
        region=region,
        era5=True,
        timeZone='America/Boise'
    )

    # --- Add quality & filters ---
    col = col.map(add_cloud_cover_roi) #Adds cloud cover over region property
    col = filter_landsat_8_9(col) #Filters to Landsat 8/9
    col = col.map(mask_clouds) #Masks clouds
    col = col.map(mask_snow_et) #Sets ET to zero where snow is present
    col = col.map(clip_to_region) #Clips to region

    # --- Composite and quality metrics ---
    et_band = col.select('ET')
    et_median = et_band.median().rename('ET_median')
    et_count = et_band.count().rename('ET_count')
    et_std = et_band.reduce(ee.Reducer.stdDev()).rename('ET_std')

    # --- Mask out low-count pixels ---
    et_masked = et_median.updateMask(et_count.gte(1))

   # Smooth only the valid pixels
    et_smoothed = et_masked.reduceNeighborhood(
    ee.Reducer.mean(),
    ee.Kernel.square(radius=1)
    ).rename('ET_smoothed')

    # Reinsert snow = 0 after smoothing
    snow = col.map(lambda img: img.select('QA_PIXEL').bitwiseAnd(1 << 5).neq(0)) \
          .max() \
          .rename('snow_mask')

    # set snow pixels to 0 but keep smoothing on everything else
    et_final = et_smoothed.where(snow.eq(1), 0)


    # --- Combine all useful bands ---
    final = et_smoothed.addBands(et_count).addBands(et_std) \
        .set({
            'year': year,
            'month': month,
            'system:time_start': start_date.millis(),
            'system:time_end': end_date.millis(),
            'date_range': ee.String(start_date.format('YYYY-MM-dd'))
                            .cat('_to_')
                            .cat(end_date.format('YYYY-MM-dd'))
        })

    return final



In [4]:
# =========================
# EXPORT FUNCTION
# =========================

def export_monthly_et(year):
    """Exports mean daily ET for each month of the given year to Google Drive."""

    for month in growing_months:
        print(f"📅 Processing {year}-{month:02d}...")

        et_img = get_monthly_mean_et(year, month).select('ET_smoothed').float()
        export_name = f"DryCreek_ET_{year}_{month:02d}"

        task = ee.batch.Export.image.toDrive(
            image=et_img,
            description=export_name,
            folder=export_folder,  
            fileNamePrefix=export_name,
            region=region.geometry().bounds(),
            scale=30,
            crs='EPSG:32611',
            maxPixels=1e13
        )

        task.start()
    
        print(f"Export started for {export_name}")
        while task.status()['state'] in ['READY', 'RUNNING']:
            print(f"⏳ Exporting {export_name} ...")
            time.sleep(60)

        status = task.status()
        if status['state'] == 'COMPLETED':
            print(f"✅ Export complete for {export_name}")
        else:
            print(f"❌ Export failed for {export_name}: {status}")

    print(f"✅ Year {year} export tasks submitted.")


In [5]:
#Test for one year
export_monthly_et(2020)

📅 Processing 2020-04...
Export started for DryCreek_ET_2020_04
⏳ Exporting DryCreek_ET_2020_04 ...
⏳ Exporting DryCreek_ET_2020_04 ...
✅ Export complete for DryCreek_ET_2020_04
📅 Processing 2020-05...
Export started for DryCreek_ET_2020_05
⏳ Exporting DryCreek_ET_2020_05 ...
⏳ Exporting DryCreek_ET_2020_05 ...
✅ Export complete for DryCreek_ET_2020_05
📅 Processing 2020-06...
Export started for DryCreek_ET_2020_06
⏳ Exporting DryCreek_ET_2020_06 ...
⏳ Exporting DryCreek_ET_2020_06 ...
✅ Export complete for DryCreek_ET_2020_06
📅 Processing 2020-07...
Export started for DryCreek_ET_2020_07
⏳ Exporting DryCreek_ET_2020_07 ...
⏳ Exporting DryCreek_ET_2020_07 ...
✅ Export complete for DryCreek_ET_2020_07
📅 Processing 2020-08...
Export started for DryCreek_ET_2020_08
⏳ Exporting DryCreek_ET_2020_08 ...
⏳ Exporting DryCreek_ET_2020_08 ...
✅ Export complete for DryCreek_ET_2020_08
📅 Processing 2020-09...
Export started for DryCreek_ET_2020_09
⏳ Exporting DryCreek_ET_2020_09 ...
⏳ Exporting DryC

In [6]:
# =========================
# RUN EXPORTS YEAR BY YEAR
# =========================

for year in range(start_year, end_year + 1):
    export_monthly_et(year)
    time.sleep(5)  # give EE a moment to register the task
    
    tasks = ee.data.listOperations()
    if tasks:
        last_task = tasks[-1]
        desc = last_task.get('metadata', {}).get('description', 'No description')
        state = last_task.get('metadata', {}).get('state', 'No state')
        print(f"Last task for year {year}: {desc} - {state}")

print("All export tasks submitted.")

📅 Processing 2013-04...
Export started for DryCreek_ET_2013_04
⏳ Exporting DryCreek_ET_2013_04 ...
⏳ Exporting DryCreek_ET_2013_04 ...
✅ Export complete for DryCreek_ET_2013_04
📅 Processing 2013-05...
Export started for DryCreek_ET_2013_05
⏳ Exporting DryCreek_ET_2013_05 ...
⏳ Exporting DryCreek_ET_2013_05 ...
✅ Export complete for DryCreek_ET_2013_05
📅 Processing 2013-06...
Export started for DryCreek_ET_2013_06
⏳ Exporting DryCreek_ET_2013_06 ...
⏳ Exporting DryCreek_ET_2013_06 ...
✅ Export complete for DryCreek_ET_2013_06
📅 Processing 2013-07...
Export started for DryCreek_ET_2013_07
⏳ Exporting DryCreek_ET_2013_07 ...
⏳ Exporting DryCreek_ET_2013_07 ...
✅ Export complete for DryCreek_ET_2013_07
📅 Processing 2013-08...
Export started for DryCreek_ET_2013_08
⏳ Exporting DryCreek_ET_2013_08 ...
✅ Export complete for DryCreek_ET_2013_08
📅 Processing 2013-09...
Export started for DryCreek_ET_2013_09
⏳ Exporting DryCreek_ET_2013_09 ...
✅ Export complete for DryCreek_ET_2013_09
📅 Processi